#Connect to google drive

In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#Install Openpose

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build

In [0]:
%%shell
cp /content/drive/My\ Drive/636/openpose-model/pose_iter_116000.caffemodel /content/openpose/models/face
cp /content/drive/My\ Drive/636/openpose-model/pose_iter_102000.caffemodel /content/openpose/models/hand
cp /content/drive/My\ Drive/636/openpose-model/pose_iter_584000.caffemodel /content/openpose/models/pose/body_25

In [0]:
%cd openpose/build
!cmake .. && make -j`nproc`

In [0]:
# !./build/examples/openpose/openpose.bin --video ../test.mp4 --write_json ./test/ --display 0 --number_people_max 1 --keypoint_scale 3 --face --hand --write_video ../test3.mp4

In [0]:
# def show_local_mp4_video(file_name, width=640, height=480):
#   import io
#   import base64
#   from IPython.display import HTML
#   video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
#   return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
#                         <source src="data:video/mp4;base64,{2}" type="video/mp4" />
#                       </video>'''.format(width, height, video_encoded.decode('ascii')))

# show_local_mp4_video('/content/test3.mp4', width=960, height=720)

In [0]:
# %%shell
# for entry in `ls ./drive/My\ Drive/636/data/video/pos/`; do
#   first=${entry:0:10}
#   mkdir ./drive/My\ Drive/636/data/json/pos/${first}
#   mkdir ./drive/My\ Drive/636/data/img/pos/${first}
#   echo ${first}
# done

In [0]:
# %%shell
# for entry in `ls ./drive/My\ Drive/636/data/video/neg/`; do
#   first=${entry:0:10}
#   mkdir ./drive/My\ Drive/636/data/json/neg/${first}
#   mkdir ./drive/My\ Drive/636/data/img/neg/${first}
#   echo ${first}
# done

In [0]:
# ./build/examples/openpose/openpose.bin --video ../drive/My\ Drive/636/data/video/pos/${first} --write_json ../drive/My\ Drive/636/data/json/pos/${first} --write_images ../drive/My\ Drive/636/data/img/pos/${first} --write_images_format jpg --display 0 --number_people_max 1 --keypoint_scale 3 --face --hand

In [0]:
ls /content/drive/My\ Drive/636/data/video/pos/

In [0]:
%%shell
cd /content/openpose
for entry in `ls ../drive/My\ Drive/636/data/video/pos/`; do
  first=${entry:0:10}
  echo ${first}
  ./build/examples/openpose/openpose.bin --video ../drive/My\ Drive/636/data/video/pos/$entry --write_json ../drive/My\ Drive/636/data/json/pos/${first} --write_images ../drive/My\ Drive/636/data/img/pos/${first} --write_images_format jpg --display 0 --number_people_max 1 --keypoint_scale 3 --face --hand
done

In [0]:
# %%shell
# cd openpose
# for entry in `ls ../drive/My\ Drive/636/data/video/neg/`; do
#   first=${entry:0:10}
#   ./build/examples/openpose/openpose.bin --video ../drive/My\ Drive/636/data/video/neg/$entry --write_json ../drive/My\ Drive/636/data/json/neg/${first} --write_images ../drive/My\ Drive/636/data/img/neg/${first} --write_images_format jpg --display 0 --number_people_max 1 --keypoint_scale 3 --face --hand
# done

In [0]:
# def show_local_mp4_video(file_name, width=640, height=480):
#   import io
#   import base64
#   from IPython.display import HTML
#   video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
#   return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
#                         <source src="data:video/mp4;base64,{2}" type="video/mp4" />
#                       </video>'''.format(width, height, video_encoded.decode('ascii')))

#Read input for test video

In [7]:
import json
import pandas as pd
import numpy as np
import os, os.path
import time
import csv

N_FACE = 70
N_POSE = 25
N_HAND = 21
COL_NAMES = ['vid', 'frame']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]
COL_NAMES += ['lh_' + str(i) for i in range(N_HAND * 2)]
COL_NAMES += ['rh_' + str(i) for i in range(N_HAND * 2)]

droplist_f = set([(i * 3 + 2) for i in range(N_FACE)])
droplist_p = set([(i * 3 + 2) for i in range(N_POSE)])
droplist_h = set([(i * 3 + 2) for i in range(N_HAND)])
idx_f = [i for i in range(N_FACE * 3) if (i not in droplist_f)]
idx_p = [i for i in range(N_POSE * 3) if (i not in droplist_p)]
idx_h = [i for i in range(N_HAND * 3) if (i not in droplist_h)]

# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, vid, csv_writer):
  cnt = 0
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)   
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          
          if len(data) > 0:
            row = [vid, int(name[12:-15])]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row_lh = [float(data[0]['hand_left_keypoints_2d'][j]) for j in idx_h]
            row_rh = [float(data[0]['hand_right_keypoints_2d'][j]) for j in idx_h]           
          else:
            row = [vid, int(name[12:-15])]
            row_face = [float(0) for j in idx_f]
            row_pose = [float(0) for j in idx_p]
            row_lh = [float(0) for j in idx_h]
            row_rh = [float(0) for j in idx_h]
          
          row += row_face + row_pose + row_lh + row_rh            
          row = np.array(row)
          csv_writer.writerow(row)
          cnt += 1
  print(vid, cnt)

url = ['EsxSxC09KUU', 'qNXn_kW6WtA', 'jnGqAN6YOZ0', 'PD8IWT3oa5s', 't1u9_DHs4r4', 'l3tnBOQG4zU', 'uTbEdPzsXGo', '0mc6IJd0NEw', 'LqRJSNM814g', 'IS3YZ9JSsnc', 'UUtdMuT1EBU', 'NEkWx0pmzq4']
name = ['1200_2', '1300_2', '1400_1', '1500_1', '1600_1', '1600_2', '1600_3', '1600_4', '1700_2', '1800_2', '1900_2', '2000_2']
name_dict = dict(zip(url, name))

CSV_FOLDER = '/content/drive/My Drive/csv/'
JSON_FOLDER = '/content/drive/My Drive/json/'

for folder in sorted(os.listdir(JSON_FOLDER)):
  csv_path = CSV_FOLDER + name_dict[folder] + '.csv'
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file) 
  csv_writer.writerow(COL_NAMES)
  folderpath = os.path.join(JSON_FOLDER, folder)
  json_to_csv(folderpath, csv_path, name_dict[folder], csv_writer)

FileNotFoundError: ignored

#Read input video

In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import time
import csv

N_FACE = 70
N_POSE = 25
N_HAND = 21
FOLDER = '/content/drive/My Drive/636/data/json/'
CSV_FOLDER = '/content/drive/My Drive/636/dataset/'
COL_NAMES = ['vid', 'frame', 'label']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]
COL_NAMES += ['lh_' + str(i) for i in range(N_HAND * 2)]
COL_NAMES += ['rh_' + str(i) for i in range(N_HAND * 2)]

droplist_f = set([(i * 3 + 2) for i in range(N_FACE)])
droplist_p = set([(i * 3 + 2) for i in range(N_POSE)])
droplist_h = set([(i * 3 + 2) for i in range(N_HAND)])
idx_f = [i for i in range(N_FACE * 3) if (i not in droplist_f)]
idx_p = [i for i in range(N_POSE * 3) if (i not in droplist_p)]
idx_h = [i for i in range(N_HAND * 3) if (i not in droplist_h)]

In [0]:
# collect the json output of openpose and combine them into csv
def pos_json_to_csv(folderpath, csv_path, vid, csv_writer):
  cnt = 0
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)   
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          
          if len(data) > 0:
            row = [vid, int(name[11:-15]), 1]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row_lh = [float(data[0]['hand_left_keypoints_2d'][j]) for j in idx_h]
            row_rh = [float(data[0]['hand_right_keypoints_2d'][j]) for j in idx_h]           
          else:
            row = [vid, int(name[11:-15]), 0]
            row_face = [float(0) for j in idx_f]
            row_pose = [float(0) for j in idx_p]
            row_lh = [float(0) for j in idx_h]
            row_rh = [float(0) for j in idx_h]
          
          row += row_face + row_pose + row_lh + row_rh            
          row = np.array(row)
          csv_writer.writerow(row)
          cnt += 1
  print(vid, cnt)

In [0]:
pos_csv_path = CSV_FOLDER + 'pos_tail.csv'
pos_write_file = open(pos_csv_path, 'w')
csv_writer = csv.writer(pos_write_file) 
csv_writer.writerow(COL_NAMES)
pos_folder = FOLDER + 'pos/'
for folder in sorted(os.listdir(pos_folder)):
  folderpath = os.path.join(pos_folder, folder)
  pos_json_to_csv(folderpath, pos_csv_path, folder[0:10], csv_writer)

a119-0433C 171
a119-0434C 180
a119-0435C 180
a119-0437C 210
a119-0438C 176
a119-0439C 210
a119-0440C 180
a119-0442C 210
a119-0443C 210
a119-0444C 180
a119-0445C 210
a119-0447C 210
a119-0448C 180
a119-0449C 210
a119-0450C 180
a119-0452C 210
a119-0453C 210
a119-0454C 180
a119-0455C 180
a119-0457C 180
a119-0458C 210
a119-0459C 210
a119-0460C 210
a119-0462C 210
a119-0463C 180
a119-0464C 180
a119-0465C 210
a119-0467C 180
a119-0468C 210
a119-0469C 210
a119-0470C 180
a119-0472C 180
a119-0473C 210
a119-0474C 210
a119-0475C 180
a119-0477C 210
a119-0478C 210
a119-0479C 180
a119-0480C 180
a119-0482C 210
a119-0483C 180
a119-0484C 210
a119-0485C 180
a119-0487C 180
a119-0488C 180
a119-0489C 180
a119-0490C 180
a119-0492C 210
a119-0493C 210
a119-0494C 210
a119-0495C 210
a119-0497C 180
a119-0498C 180
a119-0499C 210
a119-0500C 180
a119-0502C 180
a119-0503C 180
a119-0504C 180
a119-0505C 180
a119-0507C 180
a119-0508C 210
a119-0509C 180
a119-0510C 180
a119-0512C 180
a119-0513C 210
a119-0514C 210
a119-0515C

FileNotFoundError: ignored

In [0]:
%%shell
for entry in `ls /content/drive/My\ Drive/636/data/img/pos/`; do
  first=${entry:0:10}
  cd /content/drive/My\ Drive/636/data/img/pos/${first}
  cp "${first}_000000000000_rendered.jpg" /content/drive/My\ Drive/636/data/img/pos_sample/
  cp "${first}_000000000030_rendered.jpg" /content/drive/My\ Drive/636/data/img/pos_sample/
  cp "${first}_000000000060_rendered.jpg" /content/drive/My\ Drive/636/data/img/pos_sample/
done

In [0]:
%%shell
for entry in `ls /content/drive/My\ Drive/636/data/img/neg/`; do
  first=${entry:0:10}
  cd /content/drive/My\ Drive/636/data/img/neg/${first}
  cp "${first}_000000000000_rendered.jpg" /content/drive/My\ Drive/636/data/img/neg_sample/
  cp "${first}_000000000030_rendered.jpg" /content/drive/My\ Drive/636/data/img/neg_sample/
  cp "${first}_000000000060_rendered.jpg" /content/drive/My\ Drive/636/data/img/neg_sample/
done

cp: cannot stat 'a052-0020Y_000000000000_rendered.jpg': No such file or directory
cp: cannot stat 'a052-0020Y_000000000030_rendered.jpg': No such file or directory
cp: cannot stat 'a052-0020Y_000000000060_rendered.jpg': No such file or directory
cp: cannot stat 'a055-0011Y_000000000060_rendered.jpg': No such file or directory
cp: cannot stat 'a069-0005Y_000000000060_rendered.jpg': No such file or directory
cp: cannot stat 'a076-0015Y_000000000030_rendered.jpg': No such file or directory
cp: cannot stat 'a076-0015Y_000000000060_rendered.jpg': No such file or directory
cp: cannot stat 'a091-0012Y_000000000030_rendered.jpg': No such file or directory
cp: cannot stat 'a091-0012Y_000000000060_rendered.jpg': No such file or directory
cp: cannot stat 'a110-0005Y_000000000030_rendered.jpg': No such file or directory
cp: cannot stat 'a110-0005Y_000000000060_rendered.jpg': No such file or directory


In [0]:
ls '/content/drive/My Drive/636/dataset/'

neg.csv           neg_tail.csv  pos_tail.csv
neg_data_all.csv  pos.csv       pos_tail_tail.csv


#POS DATASET

In [0]:
pos_data = pd.read_csv(CSV_FOLDER + 'pos.csv')
pos_data1 = pd.read_csv(CSV_FOLDER + 'pos_tail.csv')
pos_data2 = pd.read_csv(CSV_FOLDER + 'pos_tail_tail.csv')

In [0]:
print(pos_data.shape, pos_data1.shape, pos_data2.shape)

(39495, 277) (54627, 277) (13479, 277)


In [0]:
pos_data1 = pos_data1.append(pos_data2, ignore_index=True)
pos_data = pos_data.append(pos_data1, ignore_index=True)

In [0]:
pos_data.to_csv(CSV_FOLDER + 'pos_data_all.csv', index=False)

In [0]:
pos_data = pd.read_csv(CSV_FOLDER + 'pos_data_all.csv')

In [0]:
drop_list = ['a119-0179C', 'a119-0414C', 'a119-0419C', 'a119-0831X', 'a119-0832X', 'a119-0821X']
drop_list += ['a119-0910X', 'a119-0915X', 'a119-0956X', 'a119-0965X', 'a119-0968X', 'a119-0970X', 'a119-0971X']
drop_list += ['a119-0' + str(i) + 'C' for i in range(212, 220)]
drop_list += ['a119-0' + str(i) + 'C' for i in range(222, 312)]

In [0]:
drop_list = set(drop_list)
pos_data.drop(pos_data.index[pos_data['vid'].isin(drop_list)], inplace = True)

In [0]:
# pos_data.drop(pos_data.index[pos_data['vid'] == 'a119-0826X'], inplace = True)

In [0]:
pos_data.drop(pos_data.columns[[0]], axis=1, inplace=True)

In [0]:
rm -r '/content/drive/My Drive/636/dataset/pos_data_all.csv'

In [0]:
pos_data.to_csv(CSV_FOLDER + 'pos_data_all.csv', index=False)

#REFINE POS

In [0]:
pos_data = pd.read_csv(CSV_FOLDER + 'pos_data_all.csv')

In [0]:
pos_data.shape

(102726, 277)

In [0]:
drop_list = ['a119-0335C', 'a119-0848X', 'a119-0850X', 'a119-0851X', 'a119-0857X', 'a119-0869X']
drop_list += ['a119-0854X', 'a119-0856X', 'a119-0863X', 'a119-0614C', 'a119-0859X', 'a119-0860X']
drop_list += ['a119-0862X', 'a119-0865X', 'a119-0853X', 'a119-0345C']
drop_data = pos_data.loc[pos_data['vid'].isin(drop_list)]
pos_data.drop(pos_data.index[pos_data['vid'].isin(drop_list)], inplace = True)

In [0]:
pos_data.drop(pos_data.index[(pos_data['vid'] == 'a119-0202C') & (pos_data['frame'] > 146) & (pos_data['frame'] < 154)], inplace = True)

In [0]:
pos_data.shape

(97771, 277)

In [0]:
pos_data.label = 1

In [0]:
pos_data.reset_index(drop=True, inplace=True)

In [0]:
array = pos_data.vid.unique()
double = set()
for i in array:
  if i >= 'a119-0730C':
    double.add(i)

In [0]:
for i in double:
  idx = list(pos_data.index[pos_data['vid'] == i].values)
  if idx[-1] - idx[0] != pos_data.frame[pos_data['vid'] == i].shape[0] - 1:
    len = pos_data.loc[pos_data['vid'] == i].frame.shape[0]
    droplist = idx[0 : len//2]
    print(i, len)
    pos_data.drop(droplist, inplace = True)

In [0]:
pos_data.shape

(74665, 277)

In [0]:
# array = pos_data.loc[pos_data['label'] == 0].vid.unique()
# for i in array:
#   print(i, pos_data.loc[(pos_data['label'] == 0) & (pos_data['vid'] == i)].frame.shape)

In [0]:
drop_data.shape

(13527, 277)

In [0]:
!rm -r '/content/drive/My Drive/636/dataset/drop_data.csv'

In [0]:
drop_data.to_csv(CSV_FOLDER + 'drop_data.csv', index=False)

In [0]:
drop_list = ['a119-0' + str(i) + 'C' for i in range(405, 670, 3)]
# drop_list += ['a119-0' + str(i) + 'C' for i in range(425, 663, 2)]
# drop_list = ['a119-0' + str(i) + 'C' for i in range(110, 831, 5)]
# drop_data2 = pos_data.loc[pos_data['vid'].isin(drop_list)]
# drop_data3 = pos_data.loc[pos_data['vid'].isin(drop_list)]
# drop_data = drop_data.append(drop_data3, ignore_index=True)
# drop_data.to_csv(CSV_FOLDER + 'drop_data.csv', index=False)
pos_data.drop(pos_data.index[pos_data['vid'].isin(drop_list)], inplace = True)

In [0]:
pos_data.reset_index(drop=True, inplace=True)

In [0]:
pos_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,rh_2,rh_3,rh_4,rh_5,rh_6,rh_7,rh_8,rh_9,rh_10,rh_11,rh_12,rh_13,rh_14,rh_15,rh_16,rh_17,rh_18,rh_19,rh_20,rh_21,rh_22,rh_23,rh_24,rh_25,rh_26,rh_27,rh_28,rh_29,rh_30,rh_31,rh_32,rh_33,rh_34,rh_35,rh_36,rh_37,rh_38,rh_39,rh_40,rh_41
0,a119-0009Y,0,1,0.184502,0.361479,0.190212,0.412629,0.210196,0.460581,0.235890,0.503738,0.287278,0.542100,0.347231,0.575667,0.415748,0.594848,0.484265,0.617226,0.552782,0.620423,0.624155,0.614029,0.689817,0.593250,0.758334,0.564478,0.823996,0.527715,0.866820,0.479762,0.886804,0.433408,0.903933,0.385456,0.915353,0.334306,0.207341,0.311928,0.255874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,a119-0009Y,1,1,0.181639,0.358275,0.190204,0.411022,0.210188,0.458975,0.235882,0.503730,0.287269,0.542092,0.347222,0.572462,0.415739,0.594840,0.484255,0.617217,0.552772,0.620414,0.624144,0.615619,0.689806,0.593241,0.758323,0.564470,0.823985,0.527706,0.866808,0.479754,0.886792,0.433400,0.903922,0.385448,0.915341,0.332700,0.207333,0.313519,0.255866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a119-0009Y,2,1,0.182108,0.358418,0.187893,0.410239,0.208139,0.457201,0.237063,0.504164,0.283341,0.543029,0.346972,0.570559,0.413497,0.596469,0.482913,0.614283,0.555222,0.620760,0.627531,0.612663,0.694055,0.593230,0.760579,0.564081,0.827104,0.528455,0.858920,0.479873,0.882058,0.431291,0.902305,0.382709,0.919659,0.330888,0.205247,0.313075,0.254417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a119-0009Y,3,1,0.181605,0.358234,0.190171,0.409390,0.210158,0.457350,0.235855,0.502111,0.284395,0.542077,0.347212,0.572452,0.415738,0.594833,0.484265,0.617214,0.552792,0.620411,0.624174,0.614016,0.689846,0.591635,0.758372,0.564458,0.824044,0.527690,0.866873,0.479730,0.886860,0.433370,0.901137,0.385411,0.915413,0.334254,0.207303,0.313472,0.255842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a119-0009Y,4,1,0.181661,0.358276,0.190226,0.411022,0.210210,0.457376,0.235903,0.503729,0.287290,0.542090,0.347242,0.574058,0.415758,0.594837,0.484274,0.617215,0.552790,0.620412,0.624161,0.614018,0.689822,0.593239,0.758338,0.564468,0.824000,0.527705,0.866822,0.478155,0.889661,0.431801,0.903935,0.383850,0.915354,0.331103,0.207355,0.313521,0.255887,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74660,a119-0830X,145,1,0.473375,0.419044,0.473375,0.469543,0.473375,0.521725,0.474321,0.565491,0.480941,0.612623,0.490398,0.656389,0.510258,0.691738,0.532954,0.715304,0.557543,0.727087,0.585914,0.722037,0.605774,0.696788,0.624688,0.658072,0.638873,0.617673,0.647384,0.570540,0.652113,0.523408,0.655896,0.471226,0.655896,0.427460,0.489452,0.417360,0.504583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74661,a119-0830X,146,1,0.474346,0.417376,0.472470,0.469138,0.472470,0.522569,0.474346,0.567651,0.479036,0.614403,0.490293,0.656146,0.509992,0.692879,0.532

In [0]:
!rm -r '/content/drive/My Drive/636/dataset/pos_data_all_new.csv'

In [0]:
pos_data.to_csv(CSV_FOLDER + 'pos_data_all_new.csv', index=False)

#Get face and hand data

In [3]:
ls /content/drive/My\ Drive/636/dataset/

dataset.csv            dataset_re.csv  neg_data_all.csv  pos_data_all_new.csv
dataset_pose_only.csv  drop_data.csv   part/


In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import csv

N_FACE = 70
N_POSE = 25
N_HAND = 21
FOLDER = '/content/drive/My Drive/636/data/json/'
CSV_FOLDER = '/content/drive/My Drive/636/dataset/'
COL_NAMES = ['vid', 'frame', 'label']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]
COL_NAMES += ['lh_' + str(i) for i in range(N_HAND * 2)]
COL_NAMES += ['rh_' + str(i) for i in range(N_HAND * 2)]

droplist_f = set([(i * 3 + 2) for i in range(N_FACE)])
droplist_p = set([(i * 3 + 2) for i in range(N_POSE)])
droplist_h = set([(i * 3 + 2) for i in range(N_HAND)])
idx_f = [i for i in range(N_FACE * 3) if (i not in droplist_f)]
idx_p = [i for i in range(N_POSE * 3) if (i not in droplist_p)]
idx_h = [i for i in range(N_HAND * 3) if (i not in droplist_h)]

In [0]:
raw_data = pd.read_csv(CSV_FOLDER + 'dataset.csv')

In [11]:
raw_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,rh_2,rh_3,rh_4,rh_5,rh_6,rh_7,rh_8,rh_9,rh_10,rh_11,rh_12,rh_13,rh_14,rh_15,rh_16,rh_17,rh_18,rh_19,rh_20,rh_21,rh_22,rh_23,rh_24,rh_25,rh_26,rh_27,rh_28,rh_29,rh_30,rh_31,rh_32,rh_33,rh_34,rh_35,rh_36,rh_37,rh_38,rh_39,rh_40,rh_41
0,a119-0009Y,0,1,0.184502,0.361479,0.190212,0.412629,0.210196,0.460581,0.235890,0.503738,0.287278,0.542100,0.347231,0.575667,0.415748,0.594848,0.484265,0.617226,0.552782,0.620423,0.624155,0.614029,0.689817,0.593250,0.758334,0.564478,0.823996,0.527715,0.866820,0.479762,0.886804,0.433408,0.903933,0.385456,0.915353,0.334306,0.207341,0.311928,0.255874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,a119-0009Y,1,1,0.181639,0.358275,0.190204,0.411022,0.210188,0.458975,0.235882,0.503730,0.287269,0.542092,0.347222,0.572462,0.415739,0.594840,0.484255,0.617217,0.552772,0.620414,0.624144,0.615619,0.689806,0.593241,0.758323,0.564470,0.823985,0.527706,0.866808,0.479754,0.886792,0.433400,0.903922,0.385448,0.915341,0.332700,0.207333,0.313519,0.255866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a119-0009Y,2,1,0.182108,0.358418,0.187893,0.410239,0.208139,0.457201,0.237063,0.504164,0.283341,0.543029,0.346972,0.570559,0.413497,0.596469,0.482913,0.614283,0.555222,0.620760,0.627531,0.612663,0.694055,0.593230,0.760579,0.564081,0.827104,0.528455,0.858920,0.479873,0.882058,0.431291,0.902305,0.382709,0.919659,0.330888,0.205247,0.313075,0.254417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a119-0009Y,3,1,0.181605,0.358234,0.190171,0.409390,0.210158,0.457350,0.235855,0.502111,0.284395,0.542077,0.347212,0.572452,0.415738,0.594833,0.484265,0.617214,0.552792,0.620411,0.624174,0.614016,0.689846,0.591635,0.758372,0.564458,0.824044,0.527690,0.866873,0.479730,0.886860,0.433370,0.901137,0.385411,0.915413,0.334254,0.207303,0.313472,0.255842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a119-0009Y,4,1,0.181661,0.358276,0.190226,0.411022,0.210210,0.457376,0.235903,0.503729,0.287290,0.542090,0.347242,0.574058,0.415758,0.594837,0.484274,0.617215,0.552790,0.620412,0.624161,0.614018,0.689822,0.593239,0.758338,0.564468,0.824000,0.527705,0.866822,0.478155,0.889661,0.431801,0.903935,0.383850,0.915354,0.331103,0.207355,0.313521,0.255887,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149097,a120-0047Y,135,0,0.359706,0.495687,0.358841,0.551095,0.360572,0.601885,0.364033,0.651136,0.375280,0.697308,0.395180,0.735786,0.417676,0.763489,0.444497,0.786576,0.472184,0.791193,0.492949,0.781958,0.507658,0.751177,0.519771,0.715777,0.529288,0.680378,0.540536,0.644979,0.547458,0.611119,0.553514,0.574181,0.553514,0.535704,0.389989,0.474140,0.408158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149098,a120-0047Y,136,0,0.359460,0.501576,0.358599,0.555138,0.360320,0.605639,0.363761,0.653079,0.375805,0.700519,0.395592,0.737247,0.419680,0.764793,0.4

#NEG_DATASET

In [0]:
neg_data.shape

(81331, 277)

In [0]:
neg_data = pd.read_csv(CSV_FOLDER + 'neg_data_all.csv')

In [0]:
neg_data.shape

(74437, 277)

In [0]:
for i in neg_data.vid.unique():
  if neg_data.index[(neg_data['vid'] == i) & (neg_data['frame'] == 30)].shape[0] > 1:
    print(i)

In [0]:
# neg_data.drop(neg_data.columns[[0]], axis=1, inplace=True)

In [0]:
drop_list = ['a043-0' + str(i) + 'Y' for i in range(10, 21)]
drop_list += ['a091-0012Y', 'a114-0017Y', 'a114-0023Y', 'a006-0001Y', 'a006-0012Y','a010-0002Y', 'a010-0016Y', 'a011-0007Y', 'a015-0007Y', 'a018-0008Y', 'a020-0004Y']
drop_list += ['a020-0006Y', 'a022-0018Y', 'a028-0007Y', 'a029-0008Y', 'a030-0005Y']
drop_list += ['a030-0012Y', 'a031-0006Y', 'a031-0014Y', 'a033-0007Y', 'a034-0012Y']
drop_list += ['a034-0015Y', 'a043-0006Y', 'a034-0016Y', 'a046-0008Y', 'a049-0013Y']
drop_list += ['a049-0014Y', 'a049-0015Y', 'a063-0016Y', 'a066-0018Y', 'a069-0015Y']
drop_list += ['a086-0017Y', 'a086-0019Y', 'a088-0003Y', 'a088-0006Y', 'a092-0001Y']
drop_list += ['a093-0003Y', 'a097-0007Y', 'a098-0009Y', 'a111-0026Y', 'a111-0035Y', 'a113-0013Y', 'a116-0009Y']
drop_list += ['a115-0006Y', 'a052-0020Y'] 

In [0]:
drop_list = set(drop_list)
neg_data.drop(neg_data.index[neg_data['vid'].isin(drop_list)], inplace = True)

In [0]:
neg_data.drop(neg_data.index[(neg_data['vid'] == 'a115-0017Y') & (neg_data['frame'] < 21)], inplace = True)

In [0]:
neg_data.drop(neg_data.index[(neg_data['vid'] == 'a116-0013Y') & (neg_data['frame'] > 115)], inplace = True)

In [0]:
neg_data.loc[neg_data['vid'] == 'a113-0018Y']

In [0]:
neg_data.shape

(74437, 277)

In [0]:
rm -r '/content/drive/My Drive/636/dataset/neg_data_all.csv'

In [0]:
neg_data.to_csv(CSV_FOLDER + 'neg_data_all.csv', index=False)

#divide to train and test

In [0]:
pos_data.shape

(74665, 277)

In [0]:
neg_data.shape

(74437, 277)

In [0]:
data = pos_data.append(neg_data, ignore_index=True)

In [0]:
data.shape

(149102, 277)

In [0]:
data

In [0]:
data.to_csv(CSV_FOLDER + 'dataset.csv', index=False)

In [0]:
dataset = pd.read_csv(CSV_FOLDER + 'dataset.csv')

In [0]:
vid = dataset.vid.unique()

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(vid, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape)

(807,) (143,)


In [0]:
X_test_data = dataset.loc[dataset['vid'].isin(X_test)]
X_test_data = X_test_data.reset_index()
X_test_data.drop(X_test_data.columns[[0]], axis=1, inplace=True)

In [0]:
X_test_data.shape

(23037, 277)

In [0]:
X_test_data.shape[0] / dataset.shape[0]

0.15450496975225014

In [0]:
X_test_data.index[X_test_data['label'] == 1].shape[0] / X_test_data.index[X_test_data['label'] == 0].shape[0]

1.0814058547162992